In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
print("/kaggle/inputfruits/fruits-360_dataset/fruits-360/Training/")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import tensorflow as tf
import zipfile
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os
from skimage.transform import resize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import random

In [4]:
train_dir = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training/"
test_dir = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test/"

In [5]:
# Get class names
import pathlib
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names,len(class_names))

In [6]:
# Get class names
import pathlib
data_dir = pathlib.Path(test_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names,len(class_names))

In [21]:
sel_class = random.sample(class_names,10)

In [23]:
sel = random.sample(list(class_names),10)

In [7]:
# Visualize
def view_random_image(target_dir,target_class):
  target_folder = target_dir+"/"+target_class
  random_image = random.sample(os.listdir(target_folder),1) # This will be returned as a list, so we use [0] below
  img = mpimg.imread(target_folder+"/"+random_image[0])
  fig = plt.imshow(img)
  plt.title(target_class)
  # plt.axes('off')
  fig.axes.get_xaxis().set_visible(False)
  fig.axes.get_yaxis().set_visible(False)  
  print("Image shape:",img.shape)
  return img

In [8]:
img = view_random_image(train_dir,"Apple Golden 3")

In [9]:
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

In [24]:
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size =(224,224),
                                               batch_size=256,
                                               class_mode="categorical",
                                              classes=sel)
test_data = test_datagen.flow_from_directory(directory=test_dir,
                                             target_size=(224,224),
                                             batch_size=256,
                                             class_mode="categorical",
                                            classes=sel)

In [25]:
len(train_data),len(test_data)

In [28]:
# Building the model
m1 = Sequential([
    Conv2D(filters=8,kernel_size=5,input_shape=(224,224,3),activation="relu"),
    MaxPool2D(),

    Conv2D(filters=16,kernel_size=4,activation="relu"),
    MaxPool2D(),

    Conv2D(filters=32,kernel_size=3,activation="relu"),
    tf.keras.layers.GlobalAveragePooling2D(),

    Flatten(),
    Dense(10,activation="softmax")
])
m1.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=Adam(learning_rate=0.01),
    metrics=['accuracy']
)

In [29]:
h1 = m1.fit(
    train_data,epochs=5,
    validation_data=test_data,
    steps_per_epoch=len(train_data),
    validation_steps=len(test_data)
)

In [30]:
pd.DataFrame(h1.history).plot()

### Model 2 will have the architecture given below
![](https://www.researchgate.net/profile/Jose-Naranjo-Torres/publication/341455419/figure/fig3/AS:892525732712453@1589806125572/Designed-CNN-architecture-for-fruit-classification.png)

In [32]:
# Let us try changing batch size and try with more change in model architecture
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size =(224,224),
                                               batch_size=512,
                                               class_mode="categorical",
                                              classes=sel)
test_data = test_datagen.flow_from_directory(directory=test_dir,
                                             target_size=(224,224),
                                             batch_size=512,
                                             class_mode="categorical",
                                            classes=sel)
m2 = Sequential([
    Conv2D(filters=8,kernel_size=5,input_shape=(224,224,3),activation="relu"),
    tf.keras.layers.Normalization(),
    MaxPool2D(),

    Conv2D(filters=16,kernel_size=4,activation="relu"),
    tf.keras.layers.Normalization(),
    MaxPool2D(),

    Conv2D(filters=32,kernel_size=3,activation="relu"),
    tf.keras.layers.GlobalAveragePooling2D(),

    Flatten(),
    Dense(10,activation="softmax")
])
m2.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=Adam(learning_rate=0.01),
    metrics=['accuracy']
)

In [33]:
h2 = m2.fit(
    train_data,epochs=10,
    validation_data=test_data,
    steps_per_epoch=len(train_data),
    validation_steps=len(test_data)
)

In [34]:
# Finally the model has reached above 90 accuracy scores 😇😇😇
pd.DataFrame(h2.history).plot()